In [1]:
import os
import numpy as np
import random
import pickle
import urllib.request
from PIL import Image, ImageDraw, ImageFilter, ImageEnhance
from time import localtime, strftime, time
import shutil
from datetime import datetime

In [2]:
#simple function to pickle variables for later use. save a local pickle
def save_object(obj, filename, verbose=True):
    '''Help: Given an object & filepath, store the object as a pickle for later use.'''
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    if verbose:
        print(f"File saved at {filename}")
    
#and later load the file back into a variable
def load_object(filename, verbose=True):
    '''Help: Loads something previously pickled from the provided file path.'''
    with open(filename, 'rb') as f:
        load_test = pickle.load(f)
    if verbose:
        print(f"File loaded from {filename}")
    return load_test

In [3]:
card_data_list_from_file=load_object('card_data_list.p')

File loaded from card_data_list.p


In [4]:
len(card_data_list_from_file)

38416

In [5]:
#we will generate images of ten cards
ten_cards= card_data_list_from_file[:10]

In [6]:
#ten_cards

[{'id': 'c535f7f8-a57e-4ef1-91b4-5e3281bd4407',
  'image_uri': 'https://cards.scryfall.io/large/front/c/5/c535f7f8-a57e-4ef1-91b4-5e3281bd4407.jpg?1666654357',
  'name': 'Shivan Dragon'},
 {'id': 'c9eb1309-5e60-4b8f-b63c-ebe62030db5c',
  'image_uri': 'https://cards.scryfall.io/large/front/c/9/c9eb1309-5e60-4b8f-b63c-ebe62030db5c.jpg?1666654314',
  'name': "Mishra's Factory"},
 {'id': '7dae100a-ea1d-45f3-b47c-b278cda462ad',
  'image_uri': 'https://cards.scryfall.io/large/front/7/d/7dae100a-ea1d-45f3-b47c-b278cda462ad.jpg?1666655540',
  'name': 'Necropotence'},
 {'id': '9911cc02-ef9a-458b-94d2-50af92ad6628',
  'image_uri': 'https://cards.scryfall.io/large/front/9/9/9911cc02-ef9a-458b-94d2-50af92ad6628.jpg?1666654414',
  'name': "Lim-Dûl's Vault"},
 {'id': '56b80bce-1ff6-40e7-bfdd-af1ffc1918c6',
  'image_uri': 'https://cards.scryfall.io/large/front/5/6/56b80bce-1ff6-40e7-bfdd-af1ffc1918c6.jpg?1666654464',
  'name': 'Tradewind Rider'},
 {'id': '4e5b6e96-c1b8-44b8-90ab-2989e7463427',
  'ima

In [10]:
save_object(ten_cards, 'data_Of_Trained_Cards.p')

File saved at data_Of_Trained_Cards.p


In [7]:


now = datetime.now()
def current_time():
    '''Help: Returns the current time as a nice string.'''
    #return strftime("%B %d, %-I:%M%p", localtime())
    return now.strftime("%H:%M:%S")

def elapsed_time(start_time):
    '''Using seconds since epoch, determine how much time has passed since the provided float. Returns string
    with hours:minutes:seconds'''
    elapsed_seconds = time()-start_time
    h = int(elapsed_seconds/3600)
    m = int((elapsed_seconds-h*3600)/60)
    s = int((elapsed_seconds-m*60)-h*3600)
    return f'{h}hr {m}m {s}s'


def zoom_rotate_img(image):
    '''Help: Randomly rotate and zoom the given PIL image degrees and return it'''
    #store initial image size
    initial_size = image.size
    #determine at random how much or little we scale the image
    scale = 0.95+random.random()*.1
    scaled_img_size = tuple([int(i*scale) for i in initial_size])


    #create a blank background with a random color and same size as intial image
    bg_color = tuple(np.random.choice(range(256),size=3))
    background = Image.new('RGB', initial_size, bg_color)

    #determine the center location to place our rotated card
    center_box = tuple((n-o)//2 for n,o in zip(initial_size, scaled_img_size))

    #scale the image
    scaled_img = image.resize(scaled_img_size)

    #randomly select an angle to skew the image
    max_angle = 5
    skew_angle = random.randint(-max_angle, max_angle)
    
    #add the scaled image to our color background
    background.paste(scaled_img.rotate(skew_angle, fillcolor=bg_color,expand=1).resize(scaled_img_size), 
                     center_box)

    #potentially flip the image 180 degrees
    if random.choice([True, False]):
        background = background.rotate(180)
    
    return background

def blur_img(image):
    '''Help: Blur the given PIL image and return it'''
    return image.filter(filter=ImageFilter.BLUR)

def adjust_color(image):
    '''Help: Randomly reduce or increase the saturation of the provided image and return it'''
    converter = ImageEnhance.Color(image)
    #randomly decide to half or double the image saturation
    saturation = random.choice([.5, 1.5])
    return converter.enhance(saturation)

def adjust_contrast(image):
    '''Help: Randomly decrease or increase the contrast of the provided image and return it'''
    converter = ImageEnhance.Contrast(image)
    #randomly decide to half or double the image saturation
    contrast = random.choice([.5, 1.5])
    return converter.enhance(contrast)

def adjust_sharpness(image):
    '''Help: Randomly decrease or increase the sharpness of the provided image and return it'''
    converter = ImageEnhance.Sharpness(image)
    #randomly decide to half or double the image saturation
    sharpness = random.choice([.5, 1.5])
    return converter.enhance(sharpness)

def random_edit_img(image, distort=True, verbose=True):
    '''Help: Make poor edits to the image at random and return the finished copy. Can optionally not distort
    the image if need be.'''
    
    if distort:
        #randomly choose which editing operations to perform
        edit_permission = np.random.choice(a=[False, True], size=(4))

        #always skew the image, randomly make the other edits
        image = zoom_rotate_img(image)
        if verbose:
            print('Image skewed')
        if edit_permission[0]:
            image = blur_img(image)
            if verbose:
                print('Image blurred')    
        if edit_permission[1]:
            image = adjust_color(image)
            if verbose:
                print('Image color adjusted')
        if edit_permission[2]:
            image = adjust_contrast(image)
            if verbose:
                print('Image contrast adjusted')
        if edit_permission[3]:
            image = adjust_sharpness(image)
            if verbose:
                print('Image sharpness adjusted')
    
    return image



In [8]:
def generate_distorted_imgs(cardData, num_distortions, num_undistorted, storage_path, \
                            resize=False, img_size=(224,312)):
    '''Help: High level function to reduce the testing & training image creation process down to a single step. 
    Provide a string list of multiverse_ids, the number of distortions desired for each printing, a general 
    storage location for the image files, and the final image size desired. Creates "poorly photographed" 
    versions of each multiverse_id printing provided. Results are named based on their index in the list.'''
    
    image_size = 'large'
    
    images_created = 0
    printings_distorted = 0
    #iterate through the provided list
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

    #for multiverse_id in multiverse_id_list:
    for card in cardData:
        printings_distorted += 1
        try:
            
            #pull the image URL using the multiverse_id
            headers={'User-Agent':user_agent,} 
            #request=urllib.request.Request(image_url,None,headers)
            #image_url = modified_light_df_en[modified_light_df_en['multiverse_ids']\
                                           #  == int(multiverse_id)]['image_uris'].values[0][image_size]
            image_url =card['image_uri']
            #get the raw image file
            request=urllib.request.Request(image_url,None,headers)
            response = urllib.request.urlopen(request)
            #install PIL package to convert the response into a PIL Image object to further save it
            clean_img=Image.open(response)
            #clean_img = Image.open(urlopen(image_url,None,headers))

            #use the index instead of multiverse_id in filename
            print_index = cardData.index(card)
        except:
            print("An exception occurred")

        #now produce poorly rendered versions of the printing
        for i in range(num_distortions):
            images_created += 1
            if i < num_undistorted:
                distorted_img = random_edit_img(clean_img, distort=False, verbose=False)
            else:
                distorted_img = random_edit_img(clean_img, verbose=False)

            #potentially resize the image and save it locally #.resize((224,312))
            if resize:
                distorted_img = distorted_img.resize(img_size)
                
            distorted_img.save(f"{storage_path}/{print_index}-{i}.jpg")
            
    print(f"\n{images_created} total unique distortions saved from {printings_distorted} different printings.")
    print(f"Images stored @ '{storage_path}'\n")



def prep_images_for_network(storage_path):
    '''Help: Given a folder of distorted printings, compile all images and their labels into arrays for
    neural network processing. Returns image_array, label_array'''
    
    #initialize the empty arrays
    image_array = []
    label_array = np.array([], dtype=int)

    for subdir, dirs, files in os.walk(storage_path):
        for file in np.sort(files):
            if file.endswith('.jpg'):
                #open the image, then convert it to an array and scale values from 0-1
                image = Image.open(f"{storage_path}/{file}")
                scaled_array = np.array(image)/255

                #pull the multiverse_id from the filename
                label = int(file.split('-')[0])

                #add the data
                image_array.append(scaled_array)
                label_array = np.append(label_array, label)

    #convert image list to numpy array
    image_array = np.array(image_array)
    
    return image_array, label_array


#single code block to generate distorted images, prep them for training, and save the arrays. all work is saved 
#in a new directory that is created for this run. also saves a description of the model

def generate_img_set(image_set_name, cardData, num_distortions, resize=True, verbose=True):
    '''Help: Given appropriate parameters, generate num_distortions distorted image copies of each card in 
    multiverse_id_list. Then prep all the images for neural net training and save the resulting arrays.
    Returns model_data: ((training_images, training_labels), (testing_images, testing_labels))
    
    image_set_name: str, desired folder name of current image set
    multiverse_ids_list, list of ints, card printings to use
    num_distortions, number of warped copies of each card to create
    resize, boolean, if false, images keep 936,672 original resolution, otherwise resize to (224,312)
    verbose, boolean, if true print statements show function progress
    '''

    if verbose:
        print(f"Process started for {image_set_name} on {current_time()} ...")
        start_time = time()

    #if the folder already exists, delete it so we can start fresh
    if os.path.exists(image_set_name):
        shutil.rmtree(image_set_name)

    #now create the directory, and sub folders for image storage
    os.mkdir(image_set_name)
    os.mkdir(f'{image_set_name}/Testing')
    os.mkdir(f'{image_set_name}/Training')

    if verbose:
        print(f'Folder structure created, generating {len(cardData)*num_distortions} \
training images now ...')

    #now create images for training and testing, testing will always have two images, change here if need be
    #create the training images
    training_storage_path = f"{image_set_name}/Training"
    num_undistorted = 3
    generate_distorted_imgs(cardData, num_distortions, num_undistorted, training_storage_path, resize)

    if verbose:
        print(f"Training images finished on {current_time()}, now generating {len(cardData)*2} \
testing images ...")

    #create the testing images
    testing_storage_path = f"{image_set_name}/Testing"
    num_undistorted = 1
    generate_distorted_imgs(cardData, 2, num_undistorted, testing_storage_path, resize)

    if verbose:
        print(f"All images created and saved under {image_set_name} on {current_time()}. \n\
Formatting images and labels for neural net processing now ...")

    #now open up all the image files and store contents as arrays for the neural net
    training_images, training_labels = prep_images_for_network(training_storage_path)
    testing_images, testing_labels = prep_images_for_network(testing_storage_path)

    #save the input arrays locally for later use in case we want them
    model_data = ((training_images, training_labels), (testing_images, testing_labels))
    save_object(model_data, f'{image_set_name} Arrays.p', verbose=False)

    if verbose:
        print(f"Pre processing complete on {current_time()} after {elapsed_time(start_time)}. \
\n\nTraining & testing data saved locally ({image_set_name} Arrays.p) and ready for neural network!\n\n")

    return model_data

In [9]:
image_set_name='Image_Sample'
cardData=ten_cards
num_distortion=10
resize=True
verbose=True
demo_model_data=generate_img_set(image_set_name, cardData, num_distortion, resize, verbose)

Process started for Image_Sample on 19:17:01 ...
Folder structure created, generating 100 training images now ...

100 total unique distortions saved from 10 different printings.
Images stored @ 'Image_Sample/Training'

Training images finished on 19:17:01, now generating 20 testing images ...

20 total unique distortions saved from 10 different printings.
Images stored @ 'Image_Sample/Testing'

All images created and saved under Image_Sample on 19:17:01. 
Formatting images and labels for neural net processing now ...
Pre processing complete on 19:17:01 after 0hr 0m 7s. 

Training & testing data saved locally (Image_Sample Arrays.p) and ready for neural network!


